In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
    
train_transforms = transforms.Compose([
    
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1)),
    
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=train_transforms, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=train_transforms, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=True)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
import torch.nn as nn

class Generator(nn.Module):
    pass
class Discriminator(nn.Module):
    pass


In [ ]:

gen = Generator()
disc = Discriminator()
disc_loss = nn.BCELoss()
gen_loss = nn.BCELoss()
n_epochs = 3